We will gather a list of individuals who played quarterback in college from the years 2000-2015.  (This takes a really long time to run.)  There will have to be some cleanup that will need to take place once the data is gathered.  Once this information is gathered, we will

In [1]:
from sportsipy.ncaaf.teams import Teams, Team
from sportsipy.ncaaf.roster import Player
from datetime import datetime
from sportsipy.ncaaf.boxscore import Boxscores
import pandas as pd
year_list = [str(i) for i in range(2000,2016)]
qb_list = []
for year in year_list:
    for team in Teams(year=year):
        roster = team.roster  # Gets each team's roster
        for player in roster.players:
            try:
                if player.position == 'QB':
                    # print(player.name, year) # I CAN DELETE THIS LINE ONCE I AM READY TO RUN IT FOR THE YEARS 2002-2015
                    qb_list.append([year, player.player_id])
            except TypeError:
                break
dict = {'quarterbacks': qb_list}
qb_df = pd.DataFrame(dict)
# Saving to csv so I will not have to pull all the data again as it takes a very long time. I am using only one year for now to make sure everything works.
qb_df.to_csv('college_quarterbacks2000-2015.csv')


ReadTimeout: HTTPSConnectionPool(host='www.sports-reference.com', port=443): Read timed out. (read timeout=60)

In [ ]:
# Going to format the data to get a unique list of quarterbacks.
import re
qb_df['qb_string'] = qb_df['quarterbacks'].astype(str)

qb_df['qb_string'] = qb_df['qb_string'].apply(lambda x: re.findall('\w+-\w+-\d', x))
qb_df['qb_string'] = qb_df['qb_string'].apply("".join)
qb_df.drop_duplicates(subset=['qb_string'], inplace=True)
qb_df = qb_df.reset_index()
qb_df.head(15)


'# Going to format the data to get a unique list of quarterbacks.\nimport re\nqb_df[\'qb_string\'] = qb_df[\'quarterbacks\'].astype(str)\n\nqb_df[\'qb_string\'] = qb_df[\'qb_string\'].apply(lambda x: re.findall(\'\\w+-\\w+-\\d\', x))\nqb_df[\'qb_string\'] = qb_df[\'qb_string\'].apply("".join)\nqb_df.drop_duplicates(subset=[\'qb_string\'], inplace=True)\nqb_df = qb_df.reset_index()\nqb_df.head(15)\n'

In [ ]:
# Now that we have the unique list of player ID's, we can pull their statistical information.
master_qb_df = pd.DataFrame()

for i in range(len(qb_df)):
    df = Player(qb_df['qb_string'][i]).dataframe
    master_qb_df = master_qb_df.append(df)

master_qb_df.head(20)



"# Now that we have the unique list of player ID's, we can pull their statistical information.\nmaster_qb_df = pd.DataFrame()\n\nfor i in range(len(qb_df)):\n    df = Player(qb_df['qb_string'][i]).dataframe\n    master_qb_df = master_qb_df.append(df)\n\nmaster_qb_df.head(20)\n"

In [ ]:
# saving to a CSV due to the runtime.
# only want to keep the career stats for each quarterback
master_qb_df = master_qb_df.loc['Career']
master_qb_df.head(15)
master_qb_df.to_csv('master_qb_df.csv') 

"# saving to a CSV due to the runtime.\n# only want to keep the career stats for each quarterback\nmaster_qb_df = master_qb_df.loc['Career']\nmaster_qb_df.head(15)\nmaster_qb_df.to_csv('master_qb_df.csv') "

Now to see if we can pull NFL QB Statistics.

In [ ]:
'''from sportsipy.nfl.teams import Teams, Team
from sportsipy.nfl.roster import Player

brees = Player('BreeDr00')

brees_df = brees.dataframe
brees_df['position']'''

2001        
2002      QB
2003      QB
2004      QB
2005      QB
2006      QB
2007      QB
2008      QB
2009      QB
2010      QB
2011      QB
2012      QB
2013      QB
2014      QB
2015      QB
2016      QB
2017      QB
2018      QB
2019      QB
2020      QB
Career      
Name: position, dtype: object

In [ ]:
from sportsipy.nfl.teams import Teams, Team
from sportsipy.nfl.roster import Player, Roster
import pandas as pd
year_list = [str(i) for i in range(2000,2020)]
nfl_qb_list = []
for year in year_list:
    for team in Teams(year=year):
        roster = team.roster  # Gets each team's roster
        for player in roster.players:
            try:
                if player.position == 'QB':
                    print(player.name, year) # I CAN DELETE THIS LINE ONCE I AM READY TO RUN IT FOR THE YEARS 2002-2015
                    nfl_qb_list.append([year, player.player_id])
            except TypeError:
                break
dict = {'quarterbacks': nfl_qb_list}
nfl_qb_df = pd.DataFrame(dict)
# Saving to csv so I will not have to pull all the data again as it takes a very long time. I am using only one year for now to make sure everything works.
nfl_qb_df.to_csv('nfl_quarterbacks2000-2015.csv')


In [ ]:
# Load a spreadsheet of players who have thrown a pass in the NFL between 2000 and 2020.  The goal is to get down to a quarterback list.

nfl_pass_df = pd.read_excel('NFL_QBS_2000-2015.xlsx', sheet_name='OriginalData')
print(nfl_pass_df.shape)
nfl_pass_df = nfl_pass_df[nfl_pass_df['Pos'] != 'wr']
print(nfl_pass_df.shape)
nfl_pass_df = nfl_pass_df[(nfl_pass_df['Pos'] == 'QB') | (nfl_pass_df['Pos'] == '/qb') | (nfl_pass_df['Pos'].isnull()) | (nfl_pass_df['Pos'] == 'qb')]
nfl_pass_df = nfl_pass_df[nfl_pass_df['QBrec'].notnull()]
nfl_pass_df.shape

(2121, 32)
(2088, 32)


(1160, 32)

In [ ]:
# Get a list of players who made the Pro-Bowl (one of the measures of success)
from string import punctuation
def probowl(qb_name):
    if '*' in qb_name:
        return True
    else:
        return False
nfl_pass_df['ProBowl'] = nfl_pass_df['Player'].apply(probowl)

nfl_pro_bowl_qb = nfl_pass_df[nfl_pass_df['ProBowl'] == True]

nfl_pro_bowl_qb = nfl_pro_bowl_qb['Player'].to_list()
print(len(nfl_pro_bowl_qb))
nfl_pro_bowl_qb = [x.replace("+", "") for x in nfl_pro_bowl_qb]
nfl_pro_bowl_qb = [x.replace("*", "") for x in nfl_pro_bowl_qb]
nfl_pro_bowl_qb = list(set(nfl_pro_bowl_qb))

nfl_pro_bowl_qb

180


['Matt Hasselbeck',
 'Brad Johnson',
 'Teddy Bridgewater',
 'Brett Favre',
 'Robert Griffin III',
 'Deshaun Watson',
 'Kirk Cousins',
 'Jake Plummer',
 'Donovan McNabb',
 'Carson Wentz',
 'Andrew Luck',
 'Cam Newton',
 'Kerry Collins',
 'Ben Roethlisberger',
 'Russell Wilson',
 'Dak Prescott',
 'Kordell Stewart',
 'Peyton Manning',
 'Trent Green',
 'Lamar Jackson',
 'Andy Dalton',
 'Matt Schaub',
 'Jared Goff',
 'Kurt Warner',
 'Derek Carr',
 'Eli Manning',
 'Jay Cutler',
 'Derek Anderson',
 'Tony Romo',
 'Josh Allen',
 'Matthew Stafford',
 'Alex Smith',
 'Drew Bledsoe',
 'Jameis Winston',
 'Matt Cassel',
 'Brian Griese',
 'Steve McNair',
 'Jeff Garcia',
 'Aaron Rodgers',
 'Kyler Murray',
 'Rich Gannon',
 'Philip Rivers',
 'Vince Young',
 'Mitchell Trubisky',
 'Drew Brees',
 'Matt Ryan',
 'David Garrard',
 'Jake Delhomme',
 'Carson Palmer',
 'Nick Foles\xa0',
 'Ryan Tannehill',
 'Tom Brady',
 'Elvis Grbac',
 'Marc Bulger',
 'Daunte Culpepper',
 'Tyrod Taylor',
 'Patrick Mahomes',
 'Mic

In [ ]:
# Need a unique list of qb's in the nfl_pass_df, and need to create their player ID's.

nfl_pass_df['Player'] = nfl_pass_df['Player'].apply(lambda x: x.replace('*',''))
nfl_pass_df['Player'] = nfl_pass_df['Player'].apply(lambda x: x.replace('+',''))
# print(nfl_pass_df.shape)
nfl_pass_df = nfl_pass_df.drop_duplicates(subset='Player', keep='first')
# nfl_pass_df.shape

def create_player_id(player_string):
    combined  = player_string.split(' ')
    return combined[1][:4] + combined[0][:2] + '00'

nfl_pass_df['player_id'] = nfl_pass_df['Player'].apply(create_player_id)
nfl_pass_df['player_id']


297     SiemTr00
360     PeetRo00
401     CoviSc00
451     PennCh00
469     FryeCh00
          ...   
1736    CarrDe00
1763    MayfBa00
1776    RivePh00
1825    MurrKy00
2015    HerbJu00
Name: player_id, Length: 270, dtype: object

In [ ]:
# master NFL QB dataframe listing

master_nfl_qb_df = pd.DataFrame()
nfl_pass_df = nfl_pass_df.reset_index()


for i in range(len(nfl_pass_df)):
    df = Player(nfl_pass_df['player_id'][i]).dataframe
    try:
        df['Player'] = nfl_pass_df['Player'][i]
        print(df['Player'])
    except TypeError:
        continue
    master_nfl_qb_df = master_nfl_qb_df.append(df)
master_nfl_qb_df.to_csv('master_nfl_qb_df.csv')

2015      Trevor Siemian
2016      Trevor Siemian
2017      Trevor Siemian
2019      Trevor Siemian
Career    Trevor Siemian
Name: Player, dtype: object
1989      Rodney Peete
1990      Rodney Peete
1991      Rodney Peete
1992      Rodney Peete
1993      Rodney Peete
1994      Rodney Peete
1995      Rodney Peete
1996      Rodney Peete
1997      Rodney Peete
1998      Rodney Peete
1999      Rodney Peete
2001      Rodney Peete
2002      Rodney Peete
2003      Rodney Peete
2004      Rodney Peete
Career    Rodney Peete
Name: Player, dtype: object
1999      Scott Covington
2002      Scott Covington
Career    Scott Covington
Name: Player, dtype: object
1994      Chad Pennington
1995      Chad Pennington
1996      Chad Pennington
1997      Chad Pennington
1998      Chad Pennington
1999      Chad Pennington
Career    Chad Pennington
Name: Player, dtype: object
2005      Charlie Frye
2006      Charlie Frye
2007      Charlie Frye
2008      Charlie Frye
2009      Charlie Frye
Career    Charlie Fr

In [ ]:
master_nfl_qb_df['Player'].value_counts()

Tom Brady             23
Vinny Testaverde      22
Drew Brees            21
Brett Favre           21
Ben Roethlisberger    19
                      ..
Randy Fasani           2
Troy Smith             2
Jeff Tuel              2
Keith Null             2
Henry Burris           2
Name: Player, Length: 265, dtype: int64

In [ ]:
# filter the master nfl_qb dataframe to those players that have values counts 6 and above (5 years, plus their career stat line).
master_nfl_qb_df =  master_nfl_qb_df.groupby('Player').filter(lambda x: len(x) > 5)
print(master_nfl_qb_df.shape)
master_nfl_qb_df['Player'].value_counts()

(1847, 127)


Tom Brady             23
Vinny Testaverde      22
Drew Brees            21
Brett Favre           21
Ben Roethlisberger    19
                      ..
Chris Weinke           6
Josh Johnson           6
Sean Mannion           6
Brooks Bollinger       6
Mike McMahon           6
Name: Player, Length: 174, dtype: int64

In [ ]:
# create a column of years played for each quarterback, and then filter on their career stats
master_nfl_qb_df['years_played'] = master_nfl_qb_df.groupby(['Player'])['Player'].transform('count') - 1
master_nfl_qb_df = master_nfl_qb_df.loc['Career']

print(master_nfl_qb_df.head(15))
master_nfl_qb_df.to_csv('master_nfl_qb_df_filter.csv')

       adjusted_net_yards_per_attempt_index  \
Career                                  NaN   
Career                                 None   
Career                                  NaN   
Career                                  NaN   
Career                                 None   
Career                                 None   
Career                                  NaN   
Career                                  NaN   
Career                                  NaN   
Career                                  NaN   
Career                                  NaN   
Career                                 None   
Career                                  NaN   
Career                                  NaN   
Career                                  NaN   

       adjusted_net_yards_per_pass_attempt adjusted_yards_per_attempt  \
Career                                4.77                        5.8   
Career                                None                       None   
Career                      

In [ ]:
# going to separate the successful college quarterbacks from the non-successful quarterbacks (compare the name list of the nfl dataframe versus the master qb dataframe)

master_qb_df = pd.read_csv('master_qb_df.csv')
master_qb_df['name'].isin(master_nfl_qb_df['Player']).value_counts()

False    1364
True       74
Name: name, dtype: int64